Model sans faux profils mais qui va chercher dans une table supabase factice....compétence mise manuellement pour test 

In [4]:
# ---------------------------
# 1. Importation des bibliothèques
# ---------------------------
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
from dotenv import load_dotenv
import boto3
import io
import tempfile
import joblib
import psycopg2  # Pour se connecter à Supabase Postgres
print("Bibliothèques importées")

# ---------------------------
# 2. Charger variables d'environnement
# ---------------------------
load_dotenv('.env')

# Variables Supabase / S3
S3_ENDPOINT_URL = os.getenv("S3_ENDPOINT_URL")
S3_ACCESS_KEY_ID = os.getenv("S3_ACCESS_KEY_ID")
S3_SECRET_ACCESS_KEY = os.getenv("S3_SECRET_ACCESS_KEY")
S3_REGION = os.getenv("S3_REGION")
S3_BUCKET = "dlhybride"

# ---------------------------
# 3. Connexion S3 (Supabase Storage)
# ---------------------------
s3_client = boto3.client(
    service_name='s3',
    region_name=S3_REGION,
    endpoint_url=S3_ENDPOINT_URL,
    aws_access_key_id=S3_ACCESS_KEY_ID,
    aws_secret_access_key=S3_SECRET_ACCESS_KEY
)

# ---------------------------
# 4. Charger CSV df_competence_rome_eda_v2 depuis S3
# ---------------------------
def load_csv_from_s3(file_name, bucket_name=S3_BUCKET):
    response = s3_client.get_object(Bucket=bucket_name, Key=file_name)
    df = pd.read_csv(io.BytesIO(response["Body"].read()), dtype=str)
    return df

df_jobs = load_csv_from_s3("df_competence_rome_eda_v2.csv")
df_jobs["code_ogr_competence"] = df_jobs["code_ogr_competence"].astype(str)

# ---------------------------
# 5. Construction des mappings
# ---------------------------
skills_vocab = {code: idx for idx, code in enumerate(df_jobs['code_ogr_competence'].unique())}
skill_to_label = df_jobs.drop_duplicates('code_ogr_competence') \
                        .set_index('code_ogr_competence')['libelle_competence'].to_dict()
jobs_vocab = {rome: idx for idx, rome in enumerate(df_jobs['code_rome'].unique())}
job_labels = df_jobs.drop_duplicates('code_rome').set_index('code_rome')['libelle_rome'].to_dict()
job_to_skills = df_jobs.groupby('code_rome')['code_ogr_competence'].apply(set).to_dict()

# ---------------------------
# 6. Charger le modèle depuis S3
# ---------------------------
def load_model_from_s3(file_name="modele_final_test.pkl", bucket_name=S3_BUCKET):
    response = s3_client.get_object(Bucket=bucket_name, Key=file_name)
    model_bytes = response["Body"].read()
    with tempfile.NamedTemporaryFile(delete=False, suffix=".pkl") as tmp_file:
        tmp_file.write(model_bytes)
        tmp_model_path = tmp_file.name
    model_loaded = joblib.load(tmp_model_path)
    print(f"Modèle '{file_name}' chargé depuis S3")
    return model_loaded

model_loaded = load_model_from_s3()

# ---------------------------
# 7. Fonction pour récupérer les compétences d'un utilisateur depuis Supabase
# ---------------------------
def get_user_skills(user_id):
    """
    Récupère les compétences associées à un utilisateur depuis la table 'utilisateur'.
    """
    # Connexion à la base Supabase Postgres
    conn = psycopg2.connect(
        host=os.getenv("SUPABASE_HOST"),
        dbname=os.getenv("SUPABASE_DB"),
        user=os.getenv("SUPABASE_USER"),
        password=os.getenv("SUPABASE_PASSWORD"),
        port=os.getenv("SUPABASE_PORT", 5432)
    )
    cur = conn.cursor()
    # Exemple : on suppose que la table a des colonnes user_id et skill_code
    cur.execute("SELECT skill_code FROM utilisateur WHERE user_id = %s", (user_id,))
    rows = cur.fetchall()
    cur.close()
    conn.close()
    return [r[0] for r in rows]

# ---------------------------
# 8. Fonction de prédiction hybride
# ---------------------------
def predict_hybrid(model, input_skills, skills_vocab, job_to_skills, jobs_vocab, job_labels,
                   top_k=3, seuil=0.3, min_overlap=2):
    device = next(model.parameters()).device
    ids = [skills_vocab[s] for s in input_skills if s in skills_vocab]
    if len(ids) == 0:
        return "Indéfini (aucune compétence reconnue)"
    skills = torch.tensor(ids).unsqueeze(0).to(device)
    weights = torch.tensor([1.0 for _ in ids], dtype=torch.float).unsqueeze(0).to(device)
    v_p = model.encode_profile(skills, weights)
    all_jobs = torch.arange(len(jobs_vocab)).to(device)
    v_j = model.encode_job(all_jobs)
    scores_dl = (v_p @ v_j.T).squeeze(0)
    input_set = set(input_skills)
    overlap_scores_list = [len(input_set & set(job_to_skills.get(j, []))) for j in jobs_vocab.keys()]
    overlap_scores = torch.tensor(overlap_scores_list, device=device)
    combined_scores = 0.3 * scores_dl + 0.7 * (overlap_scores / max(1, max(overlap_scores)))
    filtered_indices = torch.arange(len(jobs_vocab), device=device)[overlap_scores >= min_overlap]
    filtered_scores = combined_scores[overlap_scores >= min_overlap]
    if len(filtered_scores) == 0:
        return "Indéfini (aucune compétence ne passe le filtre)"
    best_scores, best_idx = filtered_scores.topk(min(top_k, len(filtered_scores)))
    best_jobs = [list(jobs_vocab.keys())[i] for i in filtered_indices[best_idx]]
    lines = []
    for rome, s in zip(best_jobs, best_scores):
        libelle = job_labels.get(rome, "?")
        lines.append(f"{rome} - {libelle} - {round(float(s.detach().cpu())*100,1)}%")
    if best_scores[0] < seuil:
        return "Indéfini\n" + "\n".join(lines)
    return "\n".join(lines)

# ---------------------------
# 9. Exemple d'utilisation avec affichage détaillé des compétences
# ---------------------------
user_id = 123  # ID d'utilisateur réel dans la table Supabase
user_skills = ["100023","118788","483361","122698","122575"]#get_user_skills(user_id)

# Séparer les compétences reconnues et non reconnues
recognized_skills = [s for s in user_skills if s in skills_vocab]
unrecognized_skills = [s for s in user_skills if s not in skills_vocab]

# Préparer l'affichage avec libellé
recognized_skills_named = [f"{s} - {skill_to_label.get(s, '?')}" for s in recognized_skills]

print("="*50)
print(f"Compétences utilisateur {user_id} ({len(user_skills)} au total) :")
for s in recognized_skills_named:
    print(f"   • {s}")

if unrecognized_skills:
    print(f"\nCompétences non reconnues ({len(unrecognized_skills)}) : {unrecognized_skills}")

# Prédiction des métiers
print("\nTop-3 métiers proposés :")
prediction = predict_hybrid(
    model_loaded,
    recognized_skills,
    skills_vocab,
    job_to_skills,
    jobs_vocab,
    job_labels,
    top_k=3
)
print(prediction)




Bibliothèques importées
Modèle 'modele_final_test.pkl' chargé depuis S3
Compétences utilisateur 123 (5 au total) :
   • 100023 - Abattre un arbre
   • 118788 - Planifier des opérations de chantier
   • 483361 - Maintenir la propreté du véhicule
   • 122698 - Recenser les arbres à abattre ou à élaguer
   • 122575 - Déterminer l'abattage ou l'élagage selon la trajectoire de chute des arbres en prenant en compte l'environnement et les conditions climatiques

Top-3 métiers proposés :
A1102 - Conducteur / Conductrice d'engins d'exploitation forestière - 72.1%
A1201 - Bûcheron / Bûcheronne - 44.1%
A1209 - Elagueur / Elagueuse - 28.4%
